# Forecasting with Python ARIMA

This notebook explores and experiments with data forecasting models. The notebook is divided into sections with each section representing indicators present in the data set. Each section has the following structure:
1. Data Wrangling: the data per indicator is split off from the imported dataset and made into a series.
2. Data Exploration: the data is plotted using line plots, and seasonal decompose to visualise any trends and/or seasonality in the data.
3. Statistical Testing: the data is tested to determine parameters to be passed into ARIMA/SARIMA(X) model as needed.
4. Perseistence: a persistence model is used to forecast the data to make a bassline for expected performance.
5. Parameter Testing: the data is used for forecasting with Auto ARIMA which searches a grid space for the best model.
6. Model Testing and Evaluation: models and configuration with the best performance is used to forecast the data and evaluated with RMSE, MAPE and R-squared.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.parser import parse

# Importing the excel sheet and displaying information about the sheet.
imported_sheet = pd.read_csv("Dataset For Forecast - Sheet1.csv", parse_dates=['Period'], index_col='Period')

print(f"Columns present in data: {imported_sheet.columns} \n")

print(f"Indicators present in data: {imported_sheet['Indicator'].unique()}")

print(imported_sheet.head())

Matplotlib is building the font cache; this may take a moment.


FileNotFoundError: [Errno 2] No such file or directory: 'Dataset For Forecast - Sheet1.csv'

In [ ]:
def sheet_splitter(example_sheet, indicator):
    """
    a function that checks if an indicator is present in an example sheet,
    splits the example sheet based on the provided indicator and then,
    outputs a CSV file and determines whether time series forecasts,
    can be performed on the split data. Expects at least 15 data points,
    to pass check

    Args:
    example_sheet-pandas dataframe
    indicator-string

    Return:
    indicator_df-dataFrame
    """
    
    # Declare the indicator conditional
    indicator_conditional = example_sheet['Indicator'] == indicator
    
    # Make of copy of a slice of the original dataframe
    indicator_df = example_sheet[indicator_conditional].copy()
    
    # Determine how many data points are suitable for data forecasting
    if (len(indicator_df) >= 15):
        print(f"Indicator : {indicator}, with length: {len(indicator_df)} can be forecast")
        return indicator_df
        
    else:
        return f"""Indicator : {indicator} with length: {len(indicator_df)}, cannot be forecast, choose another indicator or check spelling"""
            
    
    

In [ ]:
# import modules
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from math import sqrt

def persistence_model(series, split_ratio):
    """
    A function that implements a walk forward persistence model and 
    evaluates it using RMSE.

    Args:s
    series- pandas time series
    split_ratio- ratio of training set to test set- float(0.1-0.9)

    Returns:
    RMSE, MAPE, R2 Score 
    """
    
    # prepare data
    X = series.values
    years = series.index
    X = X.astype('float32')
    train_size = int(len(X) * split_ratio)
    # check if series and split length can compute
    if train_size > 0:
        train, test = X[0:train_size], X[train_size:]
        train_years, test_years = years[0:train_size], years[train_size:]

        # walk-forward validation
        history = [x for x in train]
        predictions = list()
        for i in range(len(test)):
            # predict
            yhat = history[-1]
            predictions.append(yhat)
            # observation
            obs = test[i]
            date = test_years[i]
            history.append(obs)
            print(f"{date:%Y}: Predicted={yhat:.3f}, Expected={obs:.3f}")

        # report performance
        rmse = sqrt(mean_squared_error(test, predictions))
        mape = (mean_absolute_percentage_error(test, predictions)) * 100
        r_2 = r2_score(test, predictions)
        print(f'RMSE: {rmse:.3f}')
        print(f'MAPE: {mape:.3f}%')
        print(f'r2 SCORE: {r_2:.3f}')
        return rmse, mape, r_2
    else:
        return """train set not large enough, supply a longer series or increase split ratio"""

In [ ]:
def series_to_supervised(data, n_shift, dropnan=True):
    """
    a function which takes a time series and makes different lags,
    of the time series data. It then makes a dataframe containing,
    the original data and it's lags

    Args: 
    data-a time series
    n_shift-the number of lags the data should be shifted by
    dropnan-drop na values in the new dataframe, true by default

    Return
    final_supervised_data- DataFrame
    """
    # holds imported data
    df = pd.DataFrame(data)
    # holds lags as added
    cols = list()
    # holds the column names for new lags added
    names = list()

    # a loop which calls the shift function a given number of times
    for i in range(1, n_shift+1):
        cols.append(df.shift(i))
        names.append(f"Value-{i}")
    
    # the names for the created columns
    names.append("Value")
    # concatenate all the lags together
    agg = pd.concat(cols, axis=1)
    # concatenate imported data and lag list
    final_supervised_data = pd.concat([agg,data], axis=1)
    # rename columns
    final_supervised_data.columns = names

    # drop nan values if true
    if dropnan:
        final_supervised_data.dropna(inplace=True)

    return final_supervised_data


In [ ]:
# import required modules
from statsmodels.tsa.stattools import adfuller

def adf_test(series, **kwargs):
    """
    a function that performs the augmented Dick-fuller statistical test
    to determine the unit root of a time series and the whether or not
    the time series being tested is stationary.

    Args:
    series- time series
    *kw- keyword arguments for the adfuller function
    """
    # Store result of the adf test
    result = adfuller(series, **kwargs)
    
    # Print results
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))
    print(f'Result: The series is {"not " if result[1] > 0.05 else ""}stationary')
    return result[0], result[1]

In [ ]:
# import required modules
from xgboost import XGBRegressor
from numpy import asarray

def xgboost_forecast(input_X, target_y, train_ratio=0.50, **kwargs):
    """
    A function that executes a xgboost algorithm on
    provided data.
    :params input_X-the features for the model
    :params target_y-the class/target variable
    :params train-ratio-ratio of train to test, range float 0.1-0.9, default=0.50
    :params kwargs- keyword arguments for xgboost

    """
    # list to store predictions
    predictions = list()
    # years present in the dataframe
    years = input_X.index
    
    # Change the values in the data to float type
    X = input_X.values
    input_X = X.astype('float32')

    # Change the values in the data to float type
    y = target_y.values
    target_y = y.astype('float32')
    
    # split data into train and test
    train_size = int(len(input_X) * train_ratio)
    X_train, X_test = input_X[0:train_size], input_X[train_size:]
    y_train, y_test = target_y[0:train_size], target_y[train_size:]
    train_years, test_years = years[0:train_size], years[train_size:]
    
    # walk-forward validation
    history = [x for x in X_train]
    futures = [y for y in y_train]
    
    # a loop to make one step predictions
    for i in range(len(y_test)):
        # predict
        model = XGBRegressor(objective='reg:squarederror', **kwargs)
        model.fit(asarray(history), asarray(futures))
        yhat = model.predict(X_test)
        result = yhat[0]
        predictions.append(result)
        date = test_years[i]

        # observation
        obs = X_test[i]
        history.append(obs)
        futures.append(y_test[i])
        print(f"{date:%Y}: Predicted={result:.3f}, Expected={y_test[i]:.3f}")

    # report performance
    rmse = sqrt(mean_squared_error(y_test, predictions))
    mape = (mean_absolute_percentage_error(y_test, predictions)) * 100
    r_2 = r2_score(y_test, predictions)
    print(f'RMSE: {rmse:.3f}')
    print(f'MAPE: {mape:.3f}%')
    print(f'r2 SCORE: {r_2:.3f}')
    
    # plot performance
    test_list = list(y_test)
    results_df = pd.DataFrame(list(zip(test_list, predictions)), index=list(test_years),
                              columns=['test', 'predictions'])
    plt.plot(results_df['test'], label="Test Data")
    plt.plot(results_df['predictions'], color='red', label='Predictions')
    plt.title('XGBoost Predictions V Test Data')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()
    plt.show()
    return results_df


In [ ]:
# import modules
from sklearn.linear_model import LinearRegression

def linear_regression_forecast(input_X, target_y, train_ratio=0.50):
    """
    a function that executes a linear regression model on provided data.
    :params input_X-the features for the model
    :params target_y-the class/target variable
    :params train-ratio-ratio of train to test, range float 0.1-0.9, default=0.50
    
    """
    # list to make predictions
    predictions = list()
    # years present within the data
    years = input_X.index
    
    # change values in data to float type
    X = input_X.values
    input_X = X.astype('float32')

    # change values in data to float type
    y = target_y.values
    target_y = y.astype('float32')
    
    # split data into test and train sets
    train_size = int(len(input_X) * train_ratio)
    X_train, X_test = input_X[0:train_size], input_X[train_size:]
    y_train, y_test = target_y[0:train_size], target_y[train_size:]
    train_years, test_years = years[0:train_size], years[train_size:]

    # walk-forward validation
    history = [x for x in X_train]
    futures = [y for y in y_train]
    # predictions = list()
    for i in range(len(y_test)):
        # predict
        model = LinearRegression()
        model.fit(asarray(history), asarray(futures))
        yhat = model.predict(X_test)
        result = yhat[0]
        predictions.append(result)
        date = test_years[i]

        # observation
        obs = X_test[i]
        history.append(obs)
        futures.append(y_test[i])
        print(f"{date:%Y}: Predicted={result:.3f}, Expected={y_test[i]:.3f}")
    
    # report performance
    rmse = sqrt(mean_squared_error(y_test, predictions))
    mape = (mean_absolute_percentage_error(y_test, predictions)) * 100
    r_2 = r2_score(y_test, predictions)
    print(f'RMSE: {rmse:.3f}')
    print(f'MAPE: {mape:.3f}%')
    print(f'R2 SCORE: {r_2:.3f}')

    # plot performance
    test_list = list(y_test)
    results_df = pd.DataFrame(list(zip(test_list, predictions)), index=list(test_years),
                              columns=['test', 'predictions'])
    plt.plot(results_df['test'], label='Test Data')
    plt.plot(results_df['predictions'], color='red', label='Predictions')
    plt.title("Linear Regression Predictions V Test Data")
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

    return results_df

In [ ]:
# import modules
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

def ARIMA_forecast(series, p, d, q, split_ratio=0.50):
    """
    A function that makes a forecast using a ARIMA and evaluates 
    the model with RMSE.
    params: series-pandas series
    p: autoregressor term-int
    d: differencing term-int
    q: moving average term-int
    split_ratio: training set and test set ratio-float(0.1-0.9)
    """
    # make data into float type
    X = series.values
    X = X.astype('float32')
    # save dates present within data
    years = series.index

    #create train and test sets
    train_size = int(len(X) * split_ratio)
    train, test = X[0:train_size], X[train_size:]
    train_years, test_years = years[0:train_size], years[train_size:]
    
    # walk-forward validation
    history = [x for x in train]
    predictions = list()
    
    for i in range(len(test)):
        # predict
        model = ARIMA(history, order=(p,d,q))
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        date = test_years[i]

        # observation
        obs = test[i]
        history.append(obs)
        print(f"{date:%Y}: Predicted={yhat:.3f}, Expected={obs:.3f}")

    # report performance
    rmse = sqrt(mean_squared_error(test, predictions))
    mape = (mean_absolute_percentage_error(test, predictions)) * 100
    r_2 = r2_score(test, predictions)
    print(f'RMSE: {rmse:.3f}')
    print(f'MAPE: {mape:.3f}%')
    print(f'R2_SCORE: {r_2:.3f}')

    # plot performance
    test_list = list(test)
    results_df = pd.DataFrame(list(zip(test_list, predictions)), index=list(test_years),
                              columns=['test', 'predictions'])
    plt.plot(results_df['test'], label='Test Data')
    plt.plot(results_df['predictions'], color='red', label='Predictions')
    plt.title('ARIMA Predictions V Test Data')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

    return results_df

    


## Adolescent Birth rate
***
This section deals with forecasting the Adolescent birth rate, there are 30 data points for this indicator. 
The data was split 70%-30% for training and testing respectively.


### Results summary
1. Data is trend stationary; that is, it has an implicit trend.
2. ARIMA performs significantly better than the bassline.
3. Data could be improved if it is gathered on a more granular level such as monthly or daily.


In [ ]:
# Split of the adolescent birth rate indicator rows from full dataset
ABR_sheet = sheet_splitter(imported_sheet, "Adolescent birth rate")

In [ ]:
# Display columns present within Adolescent birth rate sheet
ABR_sheet.columns

In [ ]:
# Drop columns not meaningful for forecasting
ABR_forecast = ABR_sheet.drop(['Indicator', 'State', 'LGA', 'Source'], axis=1)

In [ ]:
# Line plot of Adolescent birth rates over time
ABR_forecast.plot()

#### Observations
From the plot above, a very clear downward trend can be seen in the data, with one dip around 1992 and a bump around 2010. No seasonality is apparent from this plot, it also appears that there is very little noise in the data.

In [ ]:
# Import required module
from statsmodels.tsa.seasonal import seasonal_decompose

# Multiplicative Decomposition 
result_mul = seasonal_decompose(ABR_forecast['Value'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(ABR_forecast['Value'], model='additive', extrapolate_trend='freq')

In [ ]:
# plot multiplicative decompose
fig, ax = plt.subplots(2, 2)

fig.set_figwidth(11)
fig.set_figheight(7)
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax[0,0].plot(result_mul.observed)
ax[0,0].set(xlabel='year', ylabel='Births', title='Observed')

ax[0,1].plot(result_mul.trend)
ax[0,1].set(xlabel='year', ylabel='Births', title='Trend')

ax[1,0].plot(result_mul.seasonal)
ax[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax[1,1].plot(result_mul.resid)
ax[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig.suptitle('Multiplicative Decompose', fontweight ="bold")

In [ ]:
# plot additive decompose
fig1, ax1 = plt.subplots(2, 2)

fig1.set_figwidth(11)
fig1.set_figheight(7)
fig1.subplots_adjust(hspace=0.4, wspace=0.4)

ax1[0,0].plot(result_add.observed)
ax1[0,0].set(xlabel='year', ylabel='Births', title='Observed')

ax1[0,1].plot(result_add.trend)
ax1[0,1].set(xlabel='year', ylabel='Births', title='Trend')

ax1[1,0].plot(result_add.seasonal)
ax1[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax1[1,1].plot(result_add.resid)
ax1[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig1.suptitle('Additive Decompose', fontweight ="bold")

#### Observations:
These two groups of plots above decompose the series, using multiplicative and additive methods. From the figures, it can be confirmed that there is no seasonality within the data. The decomposotion also shows us that most of the information in the series exists in the trends present within the Adolescent Birth Rate series. 

Another point to note is that both the additive and multiplicative decomposition of the series exposes the same information, and thus none has more explanatory power than the other. In the next cell we test for staionarity of the series.

In [ ]:
# Test if the series is stationary
adf_test(ABR_forecast['Value'])

In [ ]:
# Moving the data by one difference.
ABR_forecast['1difference']=ABR_forecast['Value']-ABR_forecast['Value'].shift(1)

In [ ]:
# Retest if the series is stationary
adf_test(ABR_forecast['1difference'].dropna())

In [ ]:
# Move the data by another difference
ABR_forecast['2difference']=ABR_forecast['1difference']-ABR_forecast['1difference'].shift(1)

In [ ]:
# Retest for stationarity
adf_test(ABR_forecast['2difference'].dropna())

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotting autocorrelation and partial correlation graphs to determine p,d,q for ARIMA Model
fig101=plot_acf(ABR_forecast['2difference'].dropna())
fig102=plot_pacf(ABR_forecast['2difference'].dropna(), lags=13)

#### Observations
From the Autocorrelation and Partial Correlation plots above, we can establish a range for the AR (p) and MA (q) terms. For MA we can use the autocorrelation plot to determine a range as to where the correlation between values becomes insignificant. In the plot above, this is after the zero mark, and such a good range would be 0-3 . For AR, we use the partial autocorrelation plot. In the plot we see that the correlation cuts off after lag-0, thus a good range would be from 0-3 .

In [ ]:
# Testing a Persistence Model on the Time series data
persistence_model(ABR_forecast['Value'], 0.70)

In [ ]:
from pmdarima import auto_arima
import warnings

# Auto ARIMA to determine optimal Values for ARIMA parameters

warnings.filterwarnings("ignore")

stepwise_fit = auto_arima(ABR_forecast['Value'].iloc[0:21],
                          start_p=0, start_q=0,
                          max_p=3, max_q=3, m=12, 
                          start_P=0, seasonal=False,
                          d=2, D=None, trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

stepwise_fit.summary()

In [ ]:
# Test a model with the best parameters
ARIMA_forecast(ABR_forecast['Value'], 0, 2, 0, split_ratio=0.70)

In [ ]:
# Convert time series data to supervised ML data
ABR_supervised = series_to_supervised(ABR_forecast['Value'], n_shift=2)
print(ABR_supervised.head())


In [ ]:
# Test Xgboost algorithm
xgboost_forecast(ABR_supervised.loc[:,["Value-1", "Value-2"]], ABR_supervised["Value"],
                train_ratio=0.70, learning_rate=0.1, n_estimators=23, alpha=5, max_depth=10)

In [ ]:
# Test Linear Regression algorithm
linear_regression_forecast(ABR_supervised.loc[:,["Value-1", "Value-2"]], ABR_supervised["Value"],
                           train_ratio=0.70)

### Discussion of Results
For the Adolescent Birth Rate time series, the ARIMA model above performes significantly better than a persistence(baseline) model, with *RMSE of 0.243* as opposed to *1.795* for the bassline model. The ARIMA model was trained with 21 of 30 (70%) data points, this performance could be further improved if the data is gathered on more granular level asuch as monthly/ daily

## DPT 3/ Penta 3 Coverage Rate
***
This section deals with forecasting the DPT 3 Coverage rate, there are 30 data points for this indicator. 
The data was split 70%-30% for training and testing respectively.


### Results summary
1. Data is possibly a random walk, and cannot be forecast with significant confidence.
2. ARIMA performs same as the bassline.

In [ ]:
# Split of the DPT 3/Penta 3 coverage rate indicator rows from full dataset
DPT3_df = sheet_splitter(imported_sheet, "DPT 3/Penta 3 coverage rate")

In [ ]:
# Display columns present within DPT 3/Penta 3 coverage rate sheet
DPT3_df.columns

In [ ]:
# Drop columns not meaningful for forecasting
DPT3_forecast = DPT3_df.drop(['Indicator', 'State', 'LGA', 'Source'], axis=1)

In [ ]:
# Line plot of DPT 3/Penta 3 coverage rate over time
DPT3_forecast.plot()

#### Observation
The plot has a U like trend, with the possibility of noisy data. The plot also looks jagged with large swings within the plot.

In [ ]:
# Multiplicative Decomposition 
result_mul = seasonal_decompose(DPT3_forecast['Value'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(DPT3_forecast['Value'], model='additive', extrapolate_trend='freq')


In [ ]:
# Plotting multiplicative Decomposition
fig2, ax2 = plt.subplots(2, 2)

fig2.set_figwidth(11)
fig2.set_figheight(7)
fig2.subplots_adjust(hspace=0.4, wspace=0.4)

ax2[0,0].plot(result_mul.observed)
ax2[0,0].set(xlabel='year', ylabel='Coverage rate', title='Observed')

ax2[0,1].plot(result_mul.trend)
ax2[0,1].set(xlabel='year', ylabel='Coverage rate', title='Trend')

ax2[1,0].plot(result_mul.seasonal)
ax2[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax2[1,1].plot(result_mul.resid)
ax2[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig2.suptitle('Multiplicative Decompose', fontweight ="bold")

In [ ]:
# Plotting Additive Decompose
fig3, ax3 = plt.subplots(2, 2)

fig3.set_figwidth(11)
fig3.set_figheight(7)
fig3.subplots_adjust(hspace=0.4, wspace=0.4)

ax3[0,0].plot(result_add.observed)
ax3[0,0].set(xlabel='year', ylabel='Coverage rate', title='Observed')

ax3[0,1].plot(result_add.trend)
ax3[0,1].set(xlabel='year', ylabel='Coverage rate', title='Trend')

ax3[1,0].plot(result_add.seasonal)
ax3[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax3[1,1].plot(result_add.resid)
ax3[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig3.suptitle('Additive Decompose', fontweight ="bold")

#### Observations
From the figures, it can be confirmed that there is no seasoanlity within the data. The decomposotion also shows us that most of the information in the series exists in the trends present within the DPT 3/Penta 3 coverage rate series. 
Another point to note is that both the additive and multiplicative decomposition of the series exposes the same information, and thus none has more explanatory power than the other. In the next cell we test for staionarity of the series.

In [ ]:
# Testing the data for stationarity
adf_test(DPT3_forecast['Value'])

In [ ]:
# Take a difference of the data since it wasn't stationary
DPT3_forecast['1difference']=DPT3_forecast['Value']-DPT3_forecast['Value'].shift(1)

In [ ]:
# Retest the difference
adf_test(DPT3_forecast['1difference'].dropna())

In [ ]:
# Plot autocorelation and partial autocorrelation for the differenced series to determine,
# value range for p,d,q arguments for ARIMA
fig105=plot_acf(DPT3_forecast['1difference'].dropna())
fig106=plot_pacf(DPT3_forecast['1difference'].dropna(), lags=13)

#### Observations
From the Autocorrelation and Partial Correlation plots above, we can establish a range for the AR (p) and MA (q) terms. For MA we can use the autocorrelation plot to determine a range as to where the correlation between values becomes insignificant. In the plot above, this is after the zero mark, and such a good range would be 0-3 . For AR, we use the partial autocorrelation plot. In the plot we see that the correlation cuts off after lag-0, thus a good range would be from 0-3 .

In [ ]:
# Call and evaluate a persistence model to establish a baseline
persistence_model(DPT3_forecast['Value'], 0.70)

In [ ]:
# Search for the best fit ARIMA model given the range of parameters
stepwise_fit = auto_arima(DPT3_forecast['Value'].iloc[0:21],
                          start_p=0, start_q=0,
                          max_p=3, max_q=3, m=12, 
                          start_P=0, seasonal=False,
                          d=1, D=None, trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

stepwise_fit.summary()

In [ ]:
# Test best ARIMA from search.
ARIMA_forecast(DPT3_forecast['Value'], 0, 1, 0, split_ratio=0.70)

In [ ]:
# Convert time series data to supervised ML data
DPT3_supervised = series_to_supervised(DPT3_forecast['Value'], n_shift=2)
print(DPT3_supervised.head())

In [ ]:
# Test Xgboost algorithm
xgboost_forecast(DPT3_supervised.loc[:,["Value-1", "Value-2"]], DPT3_supervised["Value"],
                train_ratio=0.70, learning_rate=0.3, n_estimators=90, alpha=10, max_depth=10)

In [ ]:
# Test Linear Regression algorithm
linear_regression_forecast(DPT3_supervised.loc[:,["Value-1", "Value-2"]], DPT3_supervised["Value"],
                           train_ratio=0.70)

### Discussion of Results
The ARIMA model which produces the best result does not have any AR and MA terms, only one term for differencing, this suggests the data might be a random walk. The best performing ARIMA model has the same performance as a persistence model, also suggests that the data is a random walk and the trained model doesn't show better explanatory power than a model based on predicting previous steps.

## Infant Mortality Rate
***
This section deals with forecasting the Infant Mortality rate, there are 30 data points for this indicator. 
The data was split 70%-30% for training and testing respectively.


### Results summary
1. Data is a smooth downward trend.
2. ARIMA performs better than the bassline.

In [ ]:
# Split of the infant mortality rate indicator rows from full dataset
IM_rate_df = sheet_splitter(imported_sheet, "Infant Mortality rate")

In [ ]:
# Display columns present within infant mortality rate sheet
IM_rate_df.columns

In [ ]:
# Drop columns not meaningful for forecasting
IM_rate_forecast = IM_rate_df.drop(['Indicator', 'State', 'LGA', 'Source'], axis=1)

In [ ]:
# Line plot of infant mortality over time
IM_rate_forecast.plot()

#### Observation
The plot has a smooth downward trend, with little or no noise in the data. 

In [ ]:
# Multiplicative Decomposition 
result_mul = seasonal_decompose(IM_rate_forecast['Value'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(IM_rate_forecast['Value'], model='additive', extrapolate_trend='freq')


In [ ]:
# Plotting Multiplicative Decomposition
fig4, ax4 = plt.subplots(2, 2)

fig4.set_figwidth(11)
fig4.set_figheight(7)
fig4.subplots_adjust(hspace=0.4, wspace=0.4)

ax4[0,0].plot(result_mul.observed)
ax4[0,0].set(xlabel='year', ylabel='IM rate', title='Observed')

ax4[0,1].plot(result_mul.trend)
ax4[0,1].set(xlabel='year', ylabel='IM rate', title='Trend')

ax4[1,0].plot(result_mul.seasonal)
ax4[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax4[1,1].plot(result_mul.resid)
ax4[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig4.suptitle('Multiplicative Decompose', fontweight ="bold")

In [ ]:
# Additive Decomposition
fig5, ax5 = plt.subplots(2, 2)

fig5.set_figwidth(11)
fig5.set_figheight(7)
fig5.subplots_adjust(hspace=0.4, wspace=0.4)

ax5[0,0].plot(result_add.observed)
ax5[0,0].set(xlabel='year', ylabel='IM Rate', title='Observed')

ax5[0,1].plot(result_add.trend)
ax5[0,1].set(xlabel='year', ylabel='IM Rate', title='Trend')

ax5[1,0].plot(result_add.seasonal)
ax5[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax5[1,1].plot(result_add.resid)
ax5[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig5.suptitle('Additive Decompose', fontweight ="bold")

#### Observations
From the figures, it can be confirmed that there is no seasoanlity within the data. The decomposotion also shows us that most of the information in the series exists in the trends present within the Infant Mortality rate series. 
Another point to note is that both the additive and multiplicative decomposition of the series exposes the same information, and thus none has more explanatory power than the other. In the next cell we test for staionarity of the series.

In [ ]:
# Test the data for stationarity
adf_test(IM_rate_forecast['Value'])

In [ ]:
# difference the series 
IM_rate_forecast['1difference']=IM_rate_forecast['Value']-IM_rate_forecast['Value'].shift(1)

In [ ]:
# Testing the difference
adf_test(IM_rate_forecast['1difference'].dropna())

In [ ]:
# Plot the partial and autocorrelation plots for the difference of the time series
fig111=plot_acf(IM_rate_forecast['1difference'].dropna())
fig112=plot_pacf(IM_rate_forecast['1difference'].dropna(), lags=13)

#### Observations
From the Autocorrelation and Partial Correlation plots above, we can establish a range for the AR (p) and MA (q) terms. For MA we can use the autocorrelation plot to determine a range as to where the correlation between values becomes insignificant. In the plot above, this is after the two lag, and such a good range would be 1-4 . For AR, we use the partial autocorrelation plot. In the plot we see that the correlation cuts off after lag-4, thus a good range would be from 2-5 .

In [ ]:
# Call and test a persistence model to establish a baseline
persistence_model(IM_rate_forecast['Value'], 0.70)

In [ ]:
# Search for the best ARIMA model for the series
stepwise_fit = auto_arima(IM_rate_forecast['Value'].iloc[0:21],
                          start_p=2, start_q=1,
                          max_p=5, max_q=4, m=12, 
                          start_P=0, seasonal=False,
                          d=1, D=None, trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

stepwise_fit.summary()

In [ ]:
# Evaluate the best ARIMA model
ARIMA_forecast(IM_rate_forecast['Value'], 3, 1, 1, split_ratio=0.70)

In [ ]:
# Convert time series data to supervised ML data
IM_rate_supervised = series_to_supervised(IM_rate_forecast['Value'], n_shift=2)
print(IM_rate_supervised.head())

In [ ]:
# Test Xgboost algorithm
xgboost_forecast(IM_rate_supervised.loc[:,["Value-1", "Value-2"]], IM_rate_supervised["Value"],
                train_ratio=0.70, learning_rate=0.1, n_estimators=27, alpha=5, max_depth=15)

In [ ]:
# Test Linear Regression algorithm
linear_regression_forecast(IM_rate_supervised.loc[:,["Value-1", "Value-2"]], IM_rate_supervised["Value"],
                           train_ratio=0.70)

### Discussion of Results.
The Time series for Infant mortality rate shows a smooth downward curve over time,and there is a significant difference in performance between the model *RMSE: 0.211* and the baseline *RMSE: 1.183* on the series. Assumption made on the data is that there is no seasonality in the data.

## Skilled attendance at delivery or birth
***
The data for Skilled attendance at delivery or birth is insufficent for forecasting, because it has a short length of 8, with some years within the data are skipped.

In [ ]:
# Split of the adolescent birth rate indicator rows from full dataset
Skilled_birth_df = sheet_splitter(imported_sheet, "Skilled attendance at delivery or birth")

## Total fertility rate
***
This section deals with forecasting the Total fertility rate, there are 30 data points for this indicator. 
The data was split 70%-30% for training and testing respectively.


### Results summary
1. Data shows a smooth downward trend.
2. ARIMA is better than the baseline.

In [ ]:
# Split of the Total fertility rate indicator rows from full dataset
TF_rate_df = sheet_splitter(imported_sheet, "Total fertility rate")

In [ ]:
# Display columns present within Total fertility rate sheet
TF_rate_df.columns

In [ ]:
# Drop columns not meaningful for forecasting
TF_rate_forecast = TF_rate_df.drop(['Indicator', 'State', 'LGA', 'Source'], axis=1)

In [ ]:
# Line plot of Total fertility rateover time
TF_rate_forecast.plot()

#### Observation
The plot is shows a slow smooth downward trend.

In [ ]:
# Multiplicative Decomposition 
result_mul = seasonal_decompose(TF_rate_forecast['Value'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(TF_rate_forecast['Value'], model='additive', extrapolate_trend='freq')

In [ ]:
# plot multiplicative decompose
fig14, ax14 = plt.subplots(2, 2)

fig14.set_figwidth(11)
fig14.set_figheight(7)
fig14.subplots_adjust(hspace=0.4, wspace=0.4)

ax14[0,0].plot(result_mul.observed)
ax14[0,0].set(xlabel='year', ylabel='T.Fertility rate', title='Observed')

ax14[0,1].plot(result_mul.trend)
ax14[0,1].set(xlabel='year', ylabel='T.Fertility rate', title='Trend')

ax14[1,0].plot(result_mul.seasonal)
ax14[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax14[1,1].plot(result_mul.resid)
ax14[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig14.suptitle('Multiplicative Decompose', fontweight ="bold")

In [ ]:
# Plot additive decompose
fig15, ax15 = plt.subplots(2, 2)

fig15.set_figwidth(11)
fig15.set_figheight(7)
fig15.subplots_adjust(hspace=0.4, wspace=0.4)

ax15[0,0].plot(result_add.observed)
ax15[0,0].set(xlabel='year', ylabel='T.Fertility rate', title='Observed')

ax15[0,1].plot(result_add.trend)
ax15[0,1].set(xlabel='year', ylabel='T.Fertility rate', title='Trend')

ax15[1,0].plot(result_add.seasonal)
ax15[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax15[1,1].plot(result_add.resid)
ax15[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig15.suptitle('Additive Decompose', fontweight ="bold")

#### Observations
From the figures, it can be confirmed that there is no seasoanlity within the data. The decomposotion also shows us that most of the information in the series exists in the trends present within the Total fertility rate series. 
Another point to note is that both the additive and multiplicative decomposition of the series exposes the same information, and thus none has more explanatory power than the other. In the next cell we test for staionarity of the series.

In [ ]:
# Test for stationarity
adf_test(TF_rate_forecast['Value'])

In [ ]:
# Take a difference of the data
TF_rate_forecast['1difference']=TF_rate_forecast['Value']-TF_rate_forecast['Value'].shift(1)

In [ ]:
# Re-test for stationarity
adf_test(TF_rate_forecast['1difference'].dropna())

In [ ]:
# Plot partial and auto correlation for the difference of total fertility rate
fig125=plot_acf(TF_rate_forecast['1difference'].dropna())
fig126=plot_pacf(TF_rate_forecast['1difference'].dropna(), lags=13)

#### Observations
From the Autocorrelation and Partial Correlation plots above, we can establish a range for the AR (p) and MA (q) terms. 
For MA we can use the autocorrelation plot to determine a range as to where the correlation between values becomes insignificant. In the plot above, this is after the two mark, and such a good range would be 1-4 . For AR, we use the partial autocorrelation plot. In the plot we see that the correlation cuts off after lag-2, thus a good range would be from 1-4 .

In [ ]:
# Evaluate a persistence model 
persistence_model(TF_rate_forecast['Value'], 0.70)

In [ ]:
# search for best parameters for ARIMA model
stepwise_fit = auto_arima(TF_rate_forecast['Value'].iloc[0:21],
                          start_p=1, start_q=1,
                          max_p=3, max_q=3, m=12, 
                          start_P=0, seasonal=False,
                          d=1, D=None, trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

stepwise_fit.summary()

In [ ]:
# Evaluate best parameters on the data
ARIMA_forecast(TF_rate_forecast['Value'], 2, 1, 0, split_ratio=0.70)

In [ ]:
# Convert time series data to supervised ML data
TF_rate_supervised = series_to_supervised(TF_rate_forecast['Value'], n_shift=2)
print(TF_rate_supervised.head())

In [ ]:
# Test Xgboost algorithm
xgboost_forecast(TF_rate_supervised.loc[:,["Value-1", "Value-2"]], TF_rate_supervised["Value"],
                train_ratio=0.70, learning_rate=0.1, n_estimators=31, alpha=5, max_depth=15)

In [ ]:
# Test Linear Regression algorithm
linear_regression_forecast(TF_rate_supervised.loc[:,["Value-1", "Value-2"]], TF_rate_supervised["Value"],
                           train_ratio=0.70)

### Discussion of Results
On the Total Fertility time series, the data shows a smooth downward trend. The ARIMA model with *RMSE of *0.007* performs better than the baseline model *RMSE: 0.059* which is significant at 0.05 p-value. There might be seasonality within the data because of the shape of the partial and autocorrelation plots.

## Under 5 Mortality rate
***
This section deals with forecasting the Under 5 Mortality rate, there are 30 data points for this indicator. 
The data was split 70%-30% for training and testing respectively.


### Results summary
1. Data shows a smooth downward trend.
2. ARIMA is better than the baseline.

In [ ]:
# Split of the Under 5 Mortality rate indicator rows from full dataset
U5_Mortality_rate_df = sheet_splitter(imported_sheet, "Under 5 Mortality rate")

In [ ]:
# Display columns present within Under 5 Mortality rate sheet
U5_Mortality_rate_df.columns

In [ ]:
# Drop columns not meaningful for forecasting
U5_mortality_rate_forecast = U5_Mortality_rate_df.drop(['Indicator', 'State', 'LGA', 'Source'], axis=1)

In [ ]:
# Line plot of Under 5 Mortality rate over time
U5_mortality_rate_forecast.plot()

#### Observation
The plot is shows a smooth downward trend.

In [ ]:
# Multiplicative Decomposition 
result_mul = seasonal_decompose(U5_mortality_rate_forecast['Value'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(U5_mortality_rate_forecast['Value'], model='additive', extrapolate_trend='freq')

In [ ]:
# Plot multiplicative decompose
fig16, ax16 = plt.subplots(2, 2)

fig16.set_figwidth(11)
fig16.set_figheight(7)
fig16.subplots_adjust(hspace=0.4, wspace=0.4)

ax16[0,0].plot(result_mul.observed)
ax16[0,0].set(xlabel='year', ylabel='U5 mortality rate', title='Observed')

ax16[0,1].plot(result_mul.trend)
ax16[0,1].set(xlabel='year', ylabel='U5 mortality rate', title='Trend')

ax16[1,0].plot(result_mul.seasonal)
ax16[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax16[1,1].plot(result_mul.resid)
ax16[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig16.suptitle('Multiplicative Decompose', fontweight ="bold")

In [ ]:
# plot additive decompose 
fig17, ax17 = plt.subplots(2, 2)

fig17.set_figwidth(11)
fig17.set_figheight(7)
fig17.subplots_adjust(hspace=0.4, wspace=0.4)

ax17[0,0].plot(result_add.observed)
ax17[0,0].set(xlabel='year', ylabel='U5 mortality rate', title='Observed')

ax17[0,1].plot(result_add.trend)
ax17[0,1].set(xlabel='year', ylabel='U5 mortality rate', title='Trend')

ax17[1,0].plot(result_add.seasonal)
ax17[1,0].set(xlabel='year', ylabel='Range', title='Seasonal')

ax17[1,1].plot(result_add.resid)
ax17[1,1].set(xlabel='year', ylabel='Range', title='Residuals')

fig17.suptitle('Additive Decompose', fontweight ="bold")

#### Observations
From the figures, it can be confirmed that there is no seasoanlity within the data. The decomposotion also shows us that most of the information in the series exists in the trends present within the Under 5 Mortality rate series. 
Another point to note is that both the additive and multiplicative decomposition of the series exposes the same information, and thus none has more explanatory power than the other. In the next cell we test for staionarity of the series.

In [ ]:
# test for stationarity
adf_test(U5_mortality_rate_forecast['Value'])

In [ ]:
# Take a difference of the data
U5_mortality_rate_forecast['1difference']=U5_mortality_rate_forecast['Value']-U5_mortality_rate_forecast['Value'].shift(1)

In [ ]:
# Test the difference for stationarity
adf_test(U5_mortality_rate_forecast['1difference'].dropna())

In [ ]:
# plot partial and autocorrelation plots for U5 mortality rate
fig129=plot_acf(U5_mortality_rate_forecast['1difference'].dropna())
fig130=plot_pacf(U5_mortality_rate_forecast['1difference'].dropna(), lags=13)

#### Observations
From the Autocorrelation and Partial Correlation plots above, we can establish a range for the AR (p) and MA (q) terms. For MA we can use the autocorrelation plot to determine a range as to where the correlation between values becomes insignificant. In the plot above, this is after the two mark, and such a good range would be 1-4 . For AR, we use the partial autocorrelation plot. In the plot we see that the correlation cuts off after lag-4, thus a good range would be from 2-5 .

In [ ]:
# Evaluate persistence (baseline) model
persistence_model(U5_mortality_rate_forecast['Value'], 0.70)

In [ ]:
# search for optimal parameters for ARIMA model
stepwise_fit = auto_arima(U5_mortality_rate_forecast['Value'].iloc[0:21],
                          start_p=2, start_q=1,
                          max_p=5, max_q=4, m=12, 
                          start_P=0, seasonal=False,
                          d=1, D=None, trace=True,
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

stepwise_fit.summary()

In [ ]:
# Evaluate best parameters on data
ARIMA_forecast(U5_mortality_rate_forecast['Value'], 4, 1, 1, split_ratio=0.70)

In [ ]:
# Convert time series data to supervised ML data
U5_mortality_rate_supervised = series_to_supervised(U5_mortality_rate_forecast['Value'], n_shift=2)
print(U5_mortality_rate_supervised.head())

In [ ]:
# Test Xgboost algorithm
xgboost_forecast(U5_mortality_rate_supervised.loc[:,["Value-1", "Value-2"]], U5_mortality_rate_supervised["Value"],
                train_ratio=0.70, learning_rate=0.1, n_estimators=25, alpha=5, max_depth=15)

In [ ]:
# Test Linear Regression algorithm
linear_regression_forecast(U5_mortality_rate_supervised.loc[:,["Value-1", "Value-2"]], U5_mortality_rate_supervised["Value"],
                           train_ratio=0.70)

### Discussion of Results
On the Total Fertility time series, the data shows a smooth downward trend. The ARIMA model with *RMSE of *0.252* performs significantly better than the baseline model *RMSE: 2.140*. There might be seasonality within the data because of the shape of the partial and autocorrelation plots.